In [1]:
from langchain_groq import ChatGroq
# os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_core.prompts import PromptTemplate
# from langchain_core.output_parsers import JsonOutputParser
# import pandas as pd
# import chromadb
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY")
)
response=llm.invoke("What is Black Clover?")
print(response.content)

Black Clover is a popular Japanese manga and anime series created by Yūki Tabata. The story takes place in a world where magic is everything, and it follows the adventures of two boys, Asta and Yuno, who are on a quest to become the next Wizard King.

In this world, magic is a part of everyday life, and people use it to perform various tasks, from simple chores to complex spells. However, Asta is born without any magical powers, which makes him an outcast in a society where magic is highly valued.

Despite his lack of magic, Asta dreams of becoming the Wizard King, a title given to the most powerful magician in the land. He is determined to prove himself and show that even without magic, he can achieve great things.

Yuno, on the other hand, is a prodigy with exceptional magical abilities. He is confident, talented, and charismatic, and he is also on a quest to become the Wizard King.

Throughout the series, Asta and Yuno face various challenges, obstacles, and enemies as they strive t

In [3]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://gopay.co.id/karier/job/data-scientist-intern-indonesia")
page_data = loader.load().pop().page_content
print(page_data)




Data Scientist Intern - Indonesia


    
					Produk
					 Transfer 
									Transfer ke akun sendiri
								
									Transfer ke orang lain
								Bayar 
									QRIS
								
									Tokopedia
								
									Gojek
								
									Tagihan & pulsa
								
									Transaksi digital
								Pinjaman 
									GoPay Pinjam
								
									GoPay Later
								Dompet Digital 
									GoPay Saldo
								
									GoPay Tabungan by Jago
								
									Gopay Syariah
								
									GoPay Coins
								
									GoPay Later
								
									Metode Lainnya
								
									Laporan pengeluaran
								GoPay Aman 
									Perlindungan akun
								
									Layanan bantuan 24/7
								
									Jaminan Saldo Kembali
								
									Perlindungan data & privasi
								Games 
									GoPay Games
								
					Merchant
					 Merchant 
									Tentang Merchant 
								
									Blog Merchant
								
					Perusahaan
					 Perusahaan 
									Tentang GoPay
								
		

In [4]:
# help(chain_extract.invoke)

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scrapped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: 'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
# It means that the output of the first model is passed as input to the second model.
# "Use this prompt (prompt_extract) as input to the model (llm) and generate the response."

res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
  "role": "Data Scientist Intern - Indonesia",
  "experience": "Intern",
  "skills": [
    "Deep Learning",
    "Computer Vision",
    "PyTorch",
    "Machine Learning",
    "Classic ML models",
    "Strong communication skills"
  ],
  "description": "We are looking for a dynamic, energetic intern who is eager to solve a variety of practical deep learning and computer vision tasks arising from different businesses in GoTo Financial. You will be working closely with the KYC data science team and attend daily standups. You will be involved in live production projects and assisting prototype development."
}
```


In [6]:
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

{'role': 'Data Scientist Intern - Indonesia', 'experience': 'Intern', 'skills': ['Deep Learning', 'Computer Vision', 'PyTorch', 'Machine Learning', 'Classic ML models', 'Strong communication skills'], 'description': 'We are looking for a dynamic, energetic intern who is eager to solve a variety of practical deep learning and computer vision tasks arising from different businesses in GoTo Financial. You will be working closely with the KYC data science team and attend daily standups. You will be involved in live production projects and assisting prototype development.'}


In [8]:
type(json_res)

dict

In [9]:
import pandas as pd

data = pd.read_csv("./portfolio_data.csv")
data.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [10]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='data_portfolio')

if not collection.count():
    # print("test")
    for _, row in data.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"Links" : row["Links"]},
                       ids=[str(uuid.uuid1())])

In [11]:
links = collection.query(query_texts=["Experience in Python and Machine Learning", "Expertise in React Native"], n_results=3)
links = links.get("metadatas")
links

[[{'Links': 'https://example.com/ml-python-portfolio'},
  {'Links': 'https://example.com/python-portfolio'},
  {'Links': 'https://example.com/pytorch-portfolio'}],
 [{'Links': 'https://example.com/react-native-portfolio'},
  {'Links': 'https://example.com/react-fastapi-portfolio'},
  {'Links': 'https://example.com/react-portfolio'}]]

In [12]:
job = json_res
job, type(job)

({'role': 'Data Scientist Intern - Indonesia',
  'experience': 'Intern',
  'skills': ['Deep Learning',
   'Computer Vision',
   'PyTorch',
   'Machine Learning',
   'Classic ML models',
   'Strong communication skills'],
  'description': 'We are looking for a dynamic, energetic intern who is eager to solve a variety of practical deep learning and computer vision tasks arising from different businesses in GoTo Financial. You will be working closely with the KYC data science team and attend daily standups. You will be involved in live production projects and assisting prototype development.'},
 dict)

In [13]:
job["skills"]

['Deep Learning',
 'Computer Vision',
 'PyTorch',
 'Machine Learning',
 'Classic ML models',
 'Strong communication skills']

In [14]:
links = collection.query(query_texts=job["skills"], n_results=2)
links = links.get("metadatas")
links

[[{'Links': 'https://example.com/ml-python-portfolio'},
  {'Links': 'https://example.com/tensorflowjs-portfolio'}],
 [{'Links': 'https://example.com/ml-python-portfolio'},
  {'Links': 'https://example.com/pytorch-portfolio'}],
 [{'Links': 'https://example.com/pytorch-portfolio'},
  {'Links': 'https://example.com/ml-python-portfolio'}],
 [{'Links': 'https://example.com/ml-python-portfolio'},
  {'Links': 'https://example.com/svelte-portfolio'}],
 [{'Links': 'https://example.com/ml-python-portfolio'},
  {'Links': 'https://example.com/cobol-portfolio'}],
 [{'Links': 'https://example.com/cobol-portfolio'},
  {'Links': 'https://example.com/ml-python-portfolio'}]]

In [15]:
type(job)

dict

In [16]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Michael, a business development executive at MichCorp. MichCorp is an innovative AI & Software Consulting company, specializing in the development of advanced technological solutions that empower businesses to optimize their operations. With a focus on leveraging cutting-edge AI technologies, MichCorp works closely with clients to enhance automation, streamline workflows, and drive business growth. Our solutions have helped numerous organizations achieve greater operational efficiency, cost savings, and scalability.

        Your job is to write a cold email to the client regarding the job mentioned above, showcasing MichCorp's expertise in meeting their needs. 
        Additionally, please include the most relevant portfolio items from the following links: {link_list}
        Remember, you are Michael, BDE at MichCorp. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)

chain_extract = prompt_email | llm
res = chain_extract.invoke(input={'job_description': str(job), 'link_list': links})
print(res.content)

Subject: Expert Data Science Solutions for GoTo Financial

Dear Hiring Manager,

I came across the Data Scientist Intern - Indonesia role at GoTo Financial and was impressed by the exciting opportunities your team is working on. As a Business Development Executive at MichCorp, I'd like to introduce our company as a potential partner in enhancing your data science capabilities.

Our team at MichCorp specializes in developing cutting-edge AI and software solutions that drive business growth and operational efficiency. We've worked with numerous clients to leverage deep learning, computer vision, and machine learning technologies, aligning perfectly with the skills required for this internship. Our expertise in PyTorch, classic ML models, and strong communication skills will ensure seamless collaboration with your KYC data science team.

I'd like to highlight some of our relevant portfolio items that demonstrate our capabilities in machine learning and PyTorch:

* https://example.com/ml-p

---

In [17]:
# import re
# # Contoh string dengan spasi ekstra
# text = "  Hello   world!  This  is   an example.   "
# text = re.sub(r'\s+', ' ', text)
# print(text)
# # Menggunakan ' '.join(text.split()) untuk menghapus spasi ekstra
# cleaned_text = ' '.join(text.split())

# print(cleaned_text)